## Use Docker to package Flask REST APP (Flask-RestPlus + gunicorn + gevent)
* create a folder my_flask
* cd my_flask
* touch app.py

* app.py contents

In [ ]:
# app.py

from flask import Flask
from flask_restplus import Resource, Api, fields
from flask_httpauth import HTTPTokenAuth
import jwt

todos = {}

authorizations = {
    'apikey': {
        'type': 'apiKey',
        'in': 'header',
        'name': 'Authorization'
    }
}

app = Flask(__name__)
api = Api(app, authorizations=authorizations, security='apikey')
app.config['SECRET_KEY'] = 'donottellanyone'
auth = HTTPTokenAuth('Bearer')

apikeys = ['key1', 'key2']
for key in apikeys:
    print('*** token for {}: {}\n'.format(key, jwt.encode({'apikey': key}, app.config['SECRET_KEY'])))

@auth.verify_token
def verify_token(token):
    try:
        key = jwt.decode(token, app.config['SECRET_KEY'])['apikey']
    except:
        return False
    
    return key in apikeys

todo_item = api.model('Todo', {'todo': fields.String('Describe your todo in a sentence.')})

@api.route('/todo/<int:todo_id>')
class Todo(Resource):
    @auth.login_required
    def get(self, todo_id):
        if not todo_id in todos:
            return {'message': 'invalid todo id'}, 401
        return {todo_id: todos[todo_id]}

@api.route('/todo')
class TodoList(Resource):
    @auth.login_required
    def get(self):
        return todos

    @api.expect(todo_item)
    @auth.login_required
    def put(self):
        todo_id = len(todos) + 1
        todos[todo_id] = api.payload['todo']
        return {todo_id: todos[todo_id]}

* gunicorn.conf.py contents

    workers = 5    # 定义同时开启的处理请求的进程数量，根据网站流量适当调整
    worker_class = "gevent"   # 采用gevent库，支持异步处理请求，提高吞吐量
    bind = "0.0.0.0:5000"    # 监听IP放宽，以便于Docker之间、Docker和宿主机之间的通信

* requirements.txt contents

    flask
    flask-restplus
    Flask-HTTPAuth
    pyjwt
    gunicorn
    gevent

* Dockerfile content

    FROM python:3
    WORKDIR /usr/src/app
    COPY requirements.txt ./
    RUN pip install --no-cache-dir -r requirements.txt
    COPY . .  
    CMD ["gunicorn", "app:app", "-c", "./gunicorn.conf.py"]

* docker build -t my_flask:v1.0 .
* docker run -d -p 5000:5000 --name my-flask1 my_flask:v1.0
* open browser http://localhost:5000 to use the Flask REST API
* docker stop my-flask1

## Use Docker to package Flask REST APP (Flask-RestPlus + NGINX + uWSGI)
* create a folder my_flask2
* cd my_flask2
* touch Dockerfile
* touch requirements.txt
* mkdir app
* touch app/main.py
* main.py shoud have the same contents as app.py above

* requirements.txt contents

    flask-restplus
    Flask-HTTPAuth
    pyjwt

* Dockerfile contents 

    FROM tiangolo/uwsgi-nginx-flask:python3.6-alpine3.7
    COPY requirements.txt ./
    RUN pip install --no-cache-dir -r requirements.txt
    COPY ./app /app

* docker build -t my_flask2:V1.0 .
* docker run --rm -d --name mycontainer -p 80:80 my_flask2:V1.0
* open browser http://localhost
* docker stop mycontainer